In [1]:
import random
import copy
MAX_LENGTH = 50

In [16]:
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
def flatten(t):
    return [item for sublist in t for item in sublist]

symbols = {
 'Implies':   '\u21D2',
 'Or':        '\u2228',
 'And':       '\u2227',
 'Not':       '\u00AC',
 'Equivalent':'\u21D4'
}


class Prover:
    
    def __init__(self, to_prove, premises=[]):
        self.conclusion= None
        self.conc_val  = -1
        self.to_prove  = to_prove._clone()
        self.proven    = False
        self.alphabet  = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
        self.vars      = self.to_prove.get_vars()
        self.init_terms= self.to_prove.get_terms()
        self.terms     = self.to_prove.get_terms()
        self.available = [v for v in alphabet if v not in self.vars]
        self.premises  = premises[:]
        self.proof_seq = premises[:]
        self.ACP       = []
        if len(self.proof_seq) == 0:
            #print('enter if')
            ax_num = random.randint(0,3)
            subs   = []
            while len(subs) < 4:
                term_num = random.randint(0, len(self.terms)-1)
                term_candidate = self.terms[term_num]
                chance = random.random()
                if chance < (1/(term_candidate.height())):
                    subs.append(self.terms[term_num])
            ax_cand = Axiom(ax_num, subs)
            self.add_step(ax_cand)
       
    def deduce(self):
        if self.to_prove.name == "Implies":
            self.ACP.append(self.to_prove.operands[0]._clone())
            assumption = self.to_prove.operands[0]._clone()
            assumption.derived_by = "ACP"
            self.proof_seq.append(assumption)
            self.to_prove = self.to_prove.operands[1]._clone()
    
    
    def thm_value(self, op1):
        if op1 == self.to_prove:
            return 50
        elif op1.includes_in_consequent(self.to_prove):
            return (1/(1+(op1.height() - self.to_prove.height())))
        else:
            return 0
            
    def add_step(self, op):
        if op not in self.proof_seq:
            self.proof_seq.append(op)
            self.proof_seq[-1].get_terms(self.terms)
            self.check_last_thm()
            if self.thm_value(op) > self.conc_val:
                self.conclusion = op
                self.conc_val   = self.thm_value(op)
    
    
    def _clone(self):
        return copy.deepcopy(self)
    
    def __repr__(self):
        rep = ""
        rep += "To Prove: " + str(self.to_prove) + "\n"
        for step in self.proof_seq:
            rep += str(step)
            rep += "\n"
        rep += "--- Therefore ---\n" + str(self.conclusion)
        return rep
    
    def fix_lines(self):
        for i in range(len(self.proof_seq)):
            self.proof_seq[i].set_line(i+1)
            
    def check_proof(self):
        for line in self.proof_seq:
            if line == self.to_prove:
                self.proven=True
                
    def check_last_thm(self):
        if self.proof_seq[-1] == self.to_prove and len(self.ACP) == 0:
            self.proven = True
        elif self.proof_seq[-1] == self.to_prove:
            antecedent    = self.ACP.pop()
            new_thm       = Operator("Implies", [antecedent, self.to_prove])._clone()
            self.to_prove = new_thm._clone()
            new_thm.derived_by = "CP"
            self.add_step(new_thm)
            self.check_last_thm()

class Operator:
    
    def __init__(self, name, operands=[], line=-1, derived_from=[], derives=[], derived_by=""):
        self.name = name
        if self.name in ['And', 'Or', 'Implies', 'Eqivalent']:
            self.arity=2
        elif self.name in ['Not']:
            self.arity=1
        elif self.name in alphabet:
            self.arity=0
        else:
            print(self.name in alphabet)
            print(self.name)
            print(type(self.name))
        self.operands = operands[:self.arity]
        self.derived_from= derived_from
        self.derives = derives
        self.line = line
        self.derived_by=derived_by
        
    def set_line(self, line_num):
        self.line=line_num
        
    def _clone(self):
        return copy.deepcopy(self)
        
    def __repr__(self):
        
        if self.line != -1:
            this_op = str(self.line) + '. '
        else:
            this_op = ""
        if len(self.operands) > 1:
            this_op += '('
            this_op += str(self.operands[0])
            this_op += symbols[self.name]
            this_op += str(self.operands[1])
            this_op += ')'
        elif len(self.operands) == 1:
            this_op += symbols[self.name]
            this_op += str(self.operands[0])
        elif len(self.operands) == 0:
            this_op += self.name
        if self.line != -1:
            if len(self.derived_from) > 0:
                this_op += '\t ' + str([str(l.line) + ' ' for l in self.derived_from])
            this_op += '\t ' + self.derived_by
        
        
        return this_op
    
    def get_fn_repr(self):
        if self.line != -1:
            this_op = str(self.line) + '. ' + self.name
        else:
            this_op = self.name
        if len(self.operands) > 0:
            this_op += '('
            for op in self.operands:
                this_op += str(op)
            this_op += ')'
        if self.line != -1:
            if len(self.derived_from) > 0:
                this_op += '\t ' + str([str(l.line) + ' ' for l in self.derived_from])
            this_op += '\t ' + self.derived_by
        return this_op
    
    def get_vars(self, var_list=[]):
        if self.arity==0:
            var_list.append(self.name)
            return var_list
        else:
            for op in self.operands:
                op.get_vars(var_list)
            return var_list
        
    def height(self):
        if self.arity == 0:
            return 1
        else:
            return 1 + max([op.height() for op in self.operands])
    
    def get_terms(self, term_list=[]):
        if self not in term_list:
            cp = self._clone()
            cp.set_line(-1)
            term_list.append(cp)
        if self.arity == 0:
            return term_list
        else:
            for op in self.operands:
                op.get_terms(term_list)
            return term_list
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.name == other.name and self.operands == other.operands
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    def includes(self, other):
        if self == other:
            return True
        elif (self.arity == 0 and self != other):
            return False
        else:
            for operand in self.operands:
                if operand.includes(other):
                    return True
            return False
        
    def includes_in_consequent(self, other):
        if self.name != 'Implies':
            return self.includes(other)
        if self == other:
            return True
        elif (self.arity == 0 and self != other):
            return False
        else:
            if self.operands[1].includes(other):
                return True
            return False
    
def Axiom(num, substitutions):
    if num == 0:
        return Operator('Implies', [substitutions[0], substitutions[0]], derived_by='Ax0')
    elif num == 1:
        return Operator('Implies', [substitutions[0], Operator('Implies', [substitutions[1], substitutions[0]] )], derived_by='Ax1')
    elif num == 2:
        return Operator('Implies', [Operator('Implies', [substitutions[0], Operator('Implies', [substitutions[1], substitutions[2]])]), Operator('Implies', [Operator('Implies', [substitutions[0], substitutions[1]]), \
                Operator('Implies', [substitutions[0], substitutions[2]])])], derived_by='Ax2')
    elif num == 3:
        return Operator('Implies', [Operator('Implies', [substitutions[0], substitutions[1]]), Operator('Implies', [Operator('Not', [substitutions[1]]), Operator('Not', [substitutions[0]])])], derived_by='Ax3')
    
def MP(op1, op2, individual):
    if op1.name == 'Implies' and op2 == op1.operands[0]:
        new_thm = Operator(op1.operands[1].name, op1.operands[1].operands, derived_from=[op1, op2], derived_by='MP')
        if new_thm not in individual.proof_seq:
            op1.derives.append(new_thm)
            op2.derives.append(new_thm)
            individual.add_step(new_thm)
            
    elif op2.name == 'Implies' and op1 == op2.operands[0]:
        new_thm = Operator(op2.operands[1].name, op2.operands[1].operands, derived_from=[op2, op1], derived_by='MP')
        if new_thm not in individual.proof_seq:
            op1.derives.append(new_thm)
            op2.derives.append(new_thm)
            individual.add_step(new_thm)


def mut(prvr):
    AXP = 1#0.5
    RMP = 0#0.02
    ORP = 0#0.02
    ANP = 0#0.01
    if random.random() < AXP and len(prvr.proof_seq) < MAX_LENGTH and not prvr.proven:  #Invoke an axiom with a term seen in 'to-prove' or any derived term
        #print('Adding Axiom')
        ax_num = random.randint(0,3)
        subs   = []
        while len(subs) < 4:
            term_num = random.randint(0, len(prvr.terms)-1)
            term_candidate = prvr.terms[term_num]
            chance = random.random()
            if chance < (1/(term_candidate.height())):
                subs.append(prvr.terms[term_num])
        ax_cand = Axiom(ax_num, subs)
        prvr.add_step(ax_cand)
        # if ax_cand not in prvr.proof_seq:
        #     prvr.proof_seq.append(Axiom(ax_num, subs))
        #     prvr.proof_seq[0].set_line(1)
        #     prvr.proof_seq[0].get_terms(prvr.terms)
        #     prvr.check_last_thm()
        #     if prvr.thm_value(ax_cand) > prvr.conc_val:
        #         prvr.conclusion = ax_cand
        #         prvr.conc_val   = prvr.thm_value(ax_cand)
        #     if prvr.thm_value(ax_cand) > prvr.conc_val:
        #         prvr.conclusion = ax_cand
        #         prvr.conc_val   = prvr.thm_value(ax_cand)
        #print('Done Adding Axiom')
        
    if random.random() < RMP and len(prvr.proof_seq) > 2:
        print('Removing Branch')
        rmv = []
        indx = random.randint(1, len(prvr.proof_seq)-1)
        rmv.append(prvr.proof_seq[indx])
        k = 0
        print('k=',k)
        print(rmv)
        print(len(rmv))
        while k < len(rmv):
            for trm in rmv[k].derives:
                rmv.append(trm)
            k += 1
            print('k=',k)
            print(rmv)
            print(len(rmv))
        for trm in rmv:
            prvr.proof_seq.remove(trm)
            for i in range(len(prvr.proof_seq)-1):
                prvr.proof_seq[i].set_line(i)
        print('Done Removing')
        
    if random.random() < ORP and len(prvr.proof_seq) < MAX_LENGTH and not prvr.proven:
        #print('Or Instructions')
        trm_num = random.randint(0, len(prvr.init_terms)-1)
        thm_num = random.randint(0, len(prvr.proof_seq) -1)
        new_thm1 = prvr.proof_seq[thm_num]._clone()
        new_thm1.set_line(-1)
        new_thm2 = prvr.init_terms[trm_num]._clone()
        new_thm  = Operator('Or', [new_thm1, new_thm2])
        prvr.proof_seq.append(new_thm)
        #print('Done with Or')
    
def Rule(name, operand):
    if name == 'Double Negation':
        if operand.name == 'Not' and operand.operands[0].name == 'Not':
            new_thm = Operator(operand.operands[0].operands[0].name, operand.operands[0].operands[0], derived_from=[operand], derived_by='Double Negation')
            return new_thm
    #if name == '

In [17]:
def fit(prvr):
    fitness = 0
    conc = prvr.conclusion
    if conc.includes_in_consequent(prvr.to_prove):
        fitness += 1/(1+(conc.height() - prvr.to_prove.height()))
    if conc == prvr.to_prove:
        fitness += 6 + MAX_LENGTH - len(prvr.proof_seq)
    return fitness
        

In [18]:
POP_SIZE   = 45
DEDUCE_P   = 0.2
ToProve    = Operator('Implies', [Operator('Implies', [Operator('b'), Operator('c')]), Operator('Implies', [Operator('Implies', [Operator('a'), Operator('b')])  ,Operator('Implies', [Operator('a'), Operator('c')]) ])])#Operator('Implies', [Operator('a'), Operator('a')])
population = []
for i in range(POP_SIZE):
    ptoprov = ToProve._clone()
    population.append(Prover(ptoprov))
generation = 1
best_fitness = -1
best_proof   = None

while generation < 5:
    print('Generation: ', generation)
    this_gen_fitness = []
    for i in range(len(population)):
        this_gen_fitness.append(fit(population[i])) #Evaluate fitness of the ith popoulation member and save it
        if (fit(population[i]) > best_fitness):       #Keep track of the best program so far
            best_fitness = fit(population[i])
            best_proof    = population[i]._clone()
            print('-----New Best Fitness!-----')
            print(best_fitness)
            print(best_proof.proof_seq)
            # logfile.write('-----New Best Fitness!-----\n')
            # logfile.write(str(best_fitness) + '\n\n')
            # logfile.write(str(best_prog))
    next_gen = []
    sorted_by_fitness = sorted(range(len(this_gen_fitness)), key=lambda x: this_gen_fitness[x], reverse=True)
    next_gen += [population[sorted_by_fitness[i]]._clone() for i in range(5)]
    #print('making next gen')
    while len(next_gen) < POP_SIZE:
        ind1 = random.randint(0, POP_SIZE-1)
        ind2 = random.randint(0, POP_SIZE-1)
        if ind1 > ind2:
            next_gen.append(population[sorted_by_fitness[ind1]]._clone())
        else:
            next_gen.append(population[sorted_by_fitness[ind2]]._clone())
    #print('made next gen')
    for ind in next_gen:
        #print('mutation')
        #print(ind)
        mut(ind)
        if random.random() < DEDUCE_P:
            ind.deduce()
        if not ind.proven and ind.conclusion.name == 'Implies':
                for i in range(len(ind.proof_seq)):
                    if ind.proof_seq[i] == ind.conclusion.operands[0]:
                        #print('Modus Ponens')
                        MP(ind.proof_seq[i], ind.conclusion, ind)
                        #ind.add_step(cand)
        #print('done mutation')
        for k in range(4):
            inst = random.randint(0, len(ind.proof_seq)-1)
            if  not ind.proven and ind.proof_seq[inst].name == 'Implies':
                for i in range(len(ind.proof_seq)):
                    if ind.proof_seq[i] == ind.proof_seq[inst].operands[0]:
                        #print('Modus Ponens')
                        MP(ind.proof_seq[i], ind.proof_seq[inst], ind)
                        #ind.add_step(cand)
                        # if cand not in ind.proof_seq:
                        #     ind.proof_seq.append(MP(ind.proof_seq[i], ind.proof_seq[inst]))
                        #     ind.check_last_thm()
                    #print('Modus Ponens Done!')
    #print('To next gen')
    population = next_gen
    generation += 1
    
for prvr in population:
    prvr.fix_lines()
print("Done")

Generation:  1
-----New Best Fitness!-----
0
[(a⇒((a⇒c)⇒a))]
-----New Best Fitness!-----
0.3333333333333333
[(a⇒(((b⇒c)⇒((a⇒b)⇒(a⇒c)))⇒a))]
Generation:  2
-----New Best Fitness!-----
0.5
[((((a⇒c)⇒c)⇒((c⇒b)⇒¬(¬b⇒¬b)))⇒((((a⇒c)⇒c)⇒(c⇒b))⇒(((a⇒c)⇒c)⇒¬(¬b⇒¬b)))), (((b⇒c)⇒((a⇒b)⇒(a⇒c)))⇒((b⇒c)⇒((a⇒b)⇒(a⇒c))))]
Generation:  3
Generation:  4
Done


In [19]:
best_proof.fix_lines()
best_proof

To Prove: ((b⇒c)⇒((a⇒b)⇒(a⇒c)))
1. ((((a⇒c)⇒c)⇒((c⇒b)⇒¬(¬b⇒¬b)))⇒((((a⇒c)⇒c)⇒(c⇒b))⇒(((a⇒c)⇒c)⇒¬(¬b⇒¬b))))	 Ax2
2. (((b⇒c)⇒((a⇒b)⇒(a⇒c)))⇒((b⇒c)⇒((a⇒b)⇒(a⇒c))))	 Ax0
--- Therefore ---
2. (((b⇒c)⇒((a⇒b)⇒(a⇒c)))⇒((b⇒c)⇒((a⇒b)⇒(a⇒c))))	 Ax0

In [12]:
best_proof.proof_seq[0].get_math_repr()

'1. (Implies(Not(Implies(ab))Not(Implies(ab)))⇒Implies(Not(Implies(ab))Not(Implies(ab))))\t Ax0'

In [19]:
population[1].thm_value(population[1].proof_seq[0])

0

In [20]:
#population[0].proof_seq.append(Axiom(2, [Operator('a'), Operator('a'), Operator('a')]))
population[0]

To Prove: Implies(Implies(ab)Implies(ac))
1. Implies(Implies(Implies(Implies(Implies(bb)b)Implies(Not(b)Not(Implies(bb))))Implies(Implies(ab)Implies(Not(b)Not(a))))Implies(Not(Implies(Implies(ab)Implies(Not(b)Not(a))))Not(Implies(Implies(Implies(bb)b)Implies(Not(b)Not(Implies(bb)))))))	 Ax3
2. Implies(Implies(Implies(Implies(Not(b)b)Implies(Not(b)Not(c)))Not(Implies(bb)))Implies(Implies(Implies(Not(b)b)Implies(Not(b)Not(c)))Not(Implies(bb))))	 Ax0
3. Implies(bc)	 ACP
4. Implies(Not(Implies(Implies(Implies(bb)b)Implies(Not(b)Not(Implies(bb)))))Not(Implies(Implies(Implies(bb)b)Implies(Not(b)Not(Implies(bb))))))	 Ax0
5. Implies(ab)	 ACP
6. Implies(bb)	 Ax0
7. a	 ACP
8. Implies(Implies(Implies(Implies(Implies(bb)Implies(bb))Implies(Implies(aImplies(bb))Not(a)))Not(b))Implies(Implies(Implies(Implies(bb)Implies(bb))Implies(Implies(aImplies(bb))Not(a)))Not(b)))	 Ax0
9. Implies(Implies(aImplies(Implies(Implies(Implies(bc)Implies(Implies(ab)Implies(ac)))Not(b))Implies(Implies(Not(b)Implies(bNot

In [17]:
population[0].proof_seq is population[1].proof_seq

True

In [7]:
best_proof

To Prove: Implies(Implies(bc)Implies(Implies(ab)Implies(ac)))
1. Implies(Implies(Implies(Implies(Implies(Implies(Implies(Implies(Implies(Implies(ab)Implies(ac))Implies(Implies(ab)Implies(ac)))Not(c))Implies(Not(Not(c))Not(Implies(Implies(Implies(ab)Implies(ac))Implies(Implies(ab)Implies(ac))))))a)Implies(Not(Implies(Implies(Implies(Implies(Implies(Not(Implies(Implies(ab)Implies(ac)))Not(Implies(Implies(Implies(ab)Implies(ac))Implies(Implies(ab)Implies(ac)))))Implies(bImplies(bc)))Implies(Implies(Implies(Not(Implies(Implies(ab)Implies(ac)))Not(Implies(Implies(Implies(ab)Implies(ac))Implies(Implies(ab)Implies(ac)))))b)Implies(Implies(Not(Implies(Implies(ab)Implies(ac)))Not(Implies(Implies(Implies(ab)Implies(ac))Implies(Implies(ab)Implies(ac)))))Implies(bc))))Implies(aa))Implies(Implies(Implies(Implies(Not(Implies(Implies(ab)Implies(ac)))Not(Implies(Implies(Implies(ab)Implies(ac))Implies(Implies(ab)Implies(ac)))))Implies(bImplies(bc)))Implies(Implies(Implies(Not(Implies(Implies(ab)Implies

In [6]:
best_proof.fix_lines()
best_proof.proof_seq[2].derives

[]

In [18]:
prf = copy.deepcopy(best_proof.proof_seq)

In [ ]:
def get_branch(line, branch=[].copy()):
    d = line.derives.copy()
    if len(d) == 0:
        branch.append(line)
        print(line)
        return branch
    else:
        for other_line in d:
            d.remove(other_line)
            get_branch(other_line, branch)
        branch.append(line)
    return branch

print(get_branch(prf[2]))

In [6]:
for i in range(0, len(population[0].proof_seq)-1):
    print(population[0].proof_seq[i])
    print(population[0].proof_seq[i].derives)
    print('\n\n')

1. Implies(aImplies(Implies(aa)a))	 Ax1
[]



1. Implies(Implies(Implies(aa)Implies(aa))Implies(Not(Implies(aa))Not(Implies(aa))))	 Ax3
[]



1. Implies(aa)	 Ax0
[Implies(aImplies(Implies(Implies(aa)a)Implies(Implies(aa)a))), Implies(aImplies(aImplies(Implies(aa)a))), Implies(Implies(aa)Implies(aImplies(Implies(aa)a))), Implies(Not(a)Not(a))]



1. Implies(Implies(aa)Implies(Not(a)Not(a)))	 Ax3
[Implies(aImplies(Implies(Implies(aa)a)Implies(Implies(aa)a))), Implies(aImplies(aImplies(Implies(aa)a))), Implies(Implies(aa)Implies(aImplies(Implies(aa)a))), Implies(Not(a)Not(a))]



Implies(Not(a)Not(a))
[Implies(aImplies(Implies(Implies(aa)a)Implies(Implies(aa)a))), Implies(aImplies(aImplies(Implies(aa)a))), Implies(Implies(aa)Implies(aImplies(Implies(aa)a))), Implies(Not(a)Not(a))]



Implies(Implies(Not(Implies(aa))Not(Implies(aa)))Implies(aImplies(Not(Implies(aa))Not(Implies(aa)))))
[Implies(aImplies(Implies(Implies(aa)a)Implies(Implies(aa)a))), Implies(aImplies(aImplies(Implies(aa)a))),

In [162]:
T = Operator('And', [Operator('Implies',[Operator('p'),Operator('q')]), Operator('r')])
T

And(Implies(pq)r)

In [124]:
T.get_terms()

[And(Implies(pq)r), Implies(pq), p, q, r]

In [163]:
R = Operator('And', [Operator('Implies',[Operator('p'),Operator('q')]), Operator('r')])
l = [R]
T in l

True

In [87]:
tst2 = Operator('Implies', [Operator('b'), Operator('a')])
tst.includes(tst2)

False

In [52]:
T.get_vars()

['p', 'q', 'r']

In [53]:
P=Prover(T)

In [54]:
P.available

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [8]:
P=Prover(Operator('Implies', [Operator('a'), Operator('a')]))

In [159]:
P.proof_seq

[1. Implies(Implies(Implies(aa)a)Implies(Not(a)Not(Implies(aa))))	 Ax3]

In [160]:
P.terms

[Implies(aa),
 a,
 Implies(Implies(Implies(aa)a)Implies(Not(a)Not(Implies(aa)))),
 Implies(Implies(aa)a),
 Implies(Not(a)Not(Implies(aa))),
 Not(a),
 Not(Implies(aa))]

In [136]:
'a' in alphabet

True

In [137]:
Operator('a').name in alphabet

True

In [20]:
T = Axiom(2, [Operator('a'), Operator('a'), Operator('a')])

In [21]:
T.height()

4

In [18]:
T

Implies(Implies(aImplies(aa))Implies(Implies(aa)Implies(aa)))

In [62]:
l = [i for i in range(20)]
sorted(l)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [30]:
T= Operator('Implies', [Operator('Implies', [Operator('b'), Operator('c')]), Operator('Implies', [Operator('Implies', [Operator('a'), Operator('b')])  ,Operator('Implies', [Operator('a'), Operator('c')]) ])])

In [31]:
T

Implies(Implies(bc)Implies(Implies(ab)Implies(ac)))

In [21]:
p1 = Prover(Operator('a'))
p2 = Prover(Operator('a'))
print(p2 == p1)
print(p2 is p1)

False
False


In [22]:
p1

[1. Implies(cc)	 Ax0, 2. Implies(Implies(aImplies(aa))Implies(Implies(aa)Implies(aa)))	 Ax2, 3. Implies(Implies(aImplies(aa))Implies(Implies(aa)Implies(aa)))	 Ax2]

In [23]:
p2

[1. Implies(cc)	 Ax0, 2. Implies(Implies(aImplies(aa))Implies(Implies(aa)Implies(aa)))	 Ax2, 3. Implies(Implies(aImplies(aa))Implies(Implies(aa)Implies(aa)))	 Ax2]

In [24]:
o1 = Operator('a')
o2 = Operator('a')
print(o2 == o1)
print(o2 is o1)

True
False
